# Creation of output files (combination of tas and siconc files)

## Objective

The goal of this notebook is to create output files expressing sea ice as a function of warming level and no longer as a function of time, for all models.

In [1]:
from watermark import watermark
%load_ext watermark
print(watermark())

Last updated: 2025-07-16T12:04:35.168556+00:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.17.2

Compiler    : GCC 13.3.0
OS          : Linux
Release     : 6.8.0-63-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



In [2]:
import sys
import os

# Chemin absolu vers le dossier codes_ORL_evaluation
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajout du chemin au sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Projet root ajouté au path:", project_root)

Projet root ajouté au path: /home/jovyan/M2_SeaIce_project


In [1]:
pip install cdo

  Using cached cdo-1.6.1-py3-none-any.whl.metadata (14 kB)
Using cached cdo-1.6.1-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from module import *

/srv/conda/envs/notebook/lib/python3.12/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [5]:
%watermark --iversions

pandas    : 2.2.3
skimage   : 0.25.0
xarray    : 2025.1.1
cdo       : 1.6.1
xesmf     : 0.8.8
numpy     : 2.0.2
seaborn   : 0.13.2
watermark : 2.5.0
cartopy   : 0.24.0
matplotlib: 3.10.0
sys       : 3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]
netCDF4   : 1.7.2
csv       : 1.0



In [5]:
scenario = 'ssp585'
racineCMIP6="/home/jovyan/private-storage"
racineMasques = "/home/jovyan/private-storage/masques/masques"

Yref1 = 1850
Yref2 = 1900

nmon = 12

Tbinmin = -.5
Tbinmax = 5.
dtbin = .25
nbin = np.int32((Tbinmax - Tbinmin)/dtbin + 1)
Tbincen = np.linspace(Tbinmin,Tbinmax,nbin)

taboo = [""]

epsilon = 0.001

def get_models(filelist):
    models = []
    for ific,fichier in enumerate(filelist):
        model = fichier.rsplit("/",1)[1].split("_")[2]
        if (model not in models): 
            models.append(model)
    models.sort()
    return(models)
    
filelist = glob.glob("/home/jovyan/private-storage/historical-Amon-tas/*.nc")
models_Thist = get_models(filelist)

filelist = glob.glob("/home/jovyan/private-storage/historical-SImon-siconc/*.nc")
models_Shist = get_models(filelist)

filelist = glob.glob("/home/jovyan/private-storage/ssp585-Amon-tas/*.nc")
models_Tscen = get_models(filelist)

filelist = glob.glob("/home/jovyan/private-storage/ssp585-SImon-siconc/*.nc")
models_Sscen = get_models(filelist)

models = []
for imod, model in enumerate(models_Thist):
    if (model not in models) and (model not in taboo) and (model in models_Shist) and (model in models_Tscen) and (model in models_Sscen):
        models.append(model)

print(models)

for imod, model in enumerate(models):

    print("Model : {} {}".format(imod,model))

    # areacella
    #
    champ = 'areacella'
    fichier = glob.glob(racineMasques+'/'+champ+'/'+champ+'_'+model+'.nc')
    if (fichier == []):
        fichier = glob.glob(racineMasques+'/pseudo_'+champ+'/'+champ+'_'+model+'.nc')
    if (fichier == []):
        prnp.int32("Il manque {} pour {} !".format(champ,model))
        quit()
    f = netCDF4.Dataset(fichier[0])
    lon = f.variables['lon'][:]
    nlon = lon.size
    lat = f.variables['lat'][:]
    nlat = lat.size
    areacella = f.variables[champ][:,:]
    aireterre = np.sum(areacella)
    # print("Aire Terre : {}".format(aireterre))
    f.close()

    # lire Thist + Tscen, extraire les temps, calculer les temperatures moyennes globales
    #
    champ = 'tas'
    fichier = glob.glob("/home/jovyan/private-storage/historical-Amon-tas/"+"/*_"+model+"_*.nc")
    fh = netCDF4.Dataset(fichier[0])
    fichier = glob.glob("/home/jovyan/private-storage/ssp585-Amon-tas/"+"/*_"+model+"_*.nc")
    fs = netCDF4.Dataset(fichier[0])
    nlon2 = fh.variables['lon'][:].size
    nlat2 = fh.variables['lat'][:].size
    if (nlon2 != nlon) or (nlat2 != nlat):
        print("{} : Nombre de longitudes ou latitude faux ! {} {} {} {}".format(model,nlon,nlon2,nlat,nlat2))
        quit()
    tname = fh.variables[champ].dimensions[0]
    timeh = fh.variables[tname][:]
    nth = timeh.shape[0]
    times = fs.variables[tname][:]
    nts = times.shape[0]
    #
    nt = nth + nts
    year = np.empty((nt),np.int32)
    month = np.empty((nt),np.int32)
    GSAT = np.empty((nt),np.float32)
    #
    t_unit = fh.variables[tname].units
    t_cal = fh.variables[tname].calendar
    tvalue = netCDF4.num2date(timeh,units = t_unit,calendar = t_cal)
    for it in range(nth):
        year[it] = tvalue[it].year
        month[it] = tvalue[it].month
        tas = fh.variables[champ][it,:,:]
        tas *= areacella
        GSAT[it] = np.sum(tas[:,:])/aireterre
    #
    t_unit = fs.variables[tname].units
    t_cal = fs.variables[tname].calendar
    tvalue = netCDF4.num2date(times,units = t_unit,calendar = t_cal)
    for it in range(nts):
        year[nth+it] = tvalue[it].year
        month[nth+it] = tvalue[it].month
        tas = fs.variables[champ][it,:,:]
        tas *= areacella
        GSAT[nth+it] = np.sum(tas[:,:])/aireterre
    #
    fh.close()
    fs.close()
    #
    iyref1 = np.argmax(year >= Yref1)
    iyref2 = nt - np.argmax(year[::-1] <= Yref2)
    GSATref = np.average(GSAT[iyref1:iyref2])
    print("  GSAT reference : {}".format(GSATref))
    GSAT[:] -= GSATref
    #
    # GSATsmooth = savgol_filter(GSAT, 12, 3) # window size arg2, polynomial order arg3
    # GSAT smoothed: exponential smoothing, tau=60 = 5 yrs
    tau = 60
    GSATsmooth = np.empty((nt),np.float32)
    GSATsmooth[0] = np.average(GSAT[0:tau])
    for it in range(1,nt):
        GSATsmooth[it] = 1./tau * GSAT[it] + (1. - 1./tau) * GSATsmooth[it-1]

    # lire Shist + Sscen
    #
    fichier = glob.glob("/home/jovyan/private-storage/historical-SImon-siconc/"+"/*_"+model+"_*.nc")
    fh = netCDF4.Dataset(fichier[0])
    fichier = glob.glob("/home/jovyan/private-storage/ssp585-SImon-siconc/"+"/*_"+model+"_*.nc")
    fs = netCDF4.Dataset(fichier[0])
    # la grille peut etre differente de celle de T
    nlon = fh.variables['siconc'].shape[2]
    nlat = fh.variables['siconc'].shape[1]
    #
    n_inbin = np.zeros((nbin,nmon),np.int32)
    sicbin = np.zeros((nbin,nmon,nlat,nlon),np.float32) 
    #
    sicread = np.empty((nlat,nlon),np.float32)
    spval = fh.variables["siconc"].missing_value
    for it in range(nt):
        if (it < nth):
            sicread = fh.variables["siconc"][it,:,:]
        else:
            sicread = fs.variables["siconc"][it-nth,:,:]
        ibin = np.argmin(np.abs(GSATsmooth[it]-Tbincen[:]))
        imon = month[it] - 1
        sicbin[ibin,imon,:,:] += sicread[:,:]
        n_inbin[ibin,imon] += 1
    #
    units = fh.variables["siconc"].units
    stdname = fh.variables["siconc"].standard_name
    longname = fh.variables["siconc"].long_name

    # normaliser les bins
    #
    for ibin in range(nbin):
        for imon in range(nmon):
            if (n_inbin[ibin,imon] > 0):
                sicbin[ibin,imon,:,:] /= n_inbin[ibin,imon]
            else:
                sicbin[ibin,imon,:,:] = spval

    # pour graphique warming level
    levels = [ 10., 25., 50., 75., 90.]
    nlev = len(levels)

    swl = np.empty((nlev,nmon,nlat,nlon),np.float32)
    for ilev,level in enumerate(levels):
        xgtl = np.where( (sicbin > level) & (np.abs(sicbin/spval -1.) > epsilon), dtbin, 0)
        swl[ilev,:,:,:] = np.sum(xgtl, axis = 0)
        swl[ilev,:,:,:] = np.where( (np.abs(sicbin[np.int32(nbin/2),:,:,:]/spval -1.) > epsilon), swl[ilev,:,:,:], spval) + Tbinmin

    # ecrire
    #
    f = netCDF4.Dataset('/home/jovyan/private-storage/SiconcWarming2/SiconcWarming_'+model+'_historical+'+scenario+'.nc',mode='w',format='NETCDF4_CLASSIC')
    #
    monname = 'month'
    month_dim = f.createDimension(monname, nmon)
    months = f.createVariable(monname, np.float32, (monname,))
    months.units = 'months'
    months.long_name = 'month'
    months.standard_name = 'month'
    months.axis = 'T'
    #
    binname = 'GSAT'
    tbin_dim = f.createDimension(binname, nbin)
    tbins = f.createVariable(binname, np.float32, (binname,))
    tbins.units = 'K'
    tbins.long_name = 'GSAT wrt. 1850-1900'
    tbins.standard_name = 'GSAT'
    tbins.axis = 'Z'
    #
    levelname = 'Level'
    lev_dim = f.createDimension(levelname, nlev)
    levs = f.createVariable(levelname, np.float32, (levelname,))
    levs.units = '%'
    levs.long_name = 'Level'
    levs.standard_name = 'Level'
    levs.axis = 'Z'
    #
    # copier les dimensions spatiales de siconc (fichier original)
    dimlist = []
    for idim in range(1,3):
        dimlist.append(fh.variables['siconc'].dimensions[idim])
    try:
        dimlist.append(fh.variables['type'].dimensions[0])
    except:
        pass
    for dname, the_dim in fh.dimensions.items():
        if (dname in dimlist):
            f.createDimension(dname, len(the_dim))
    # copier les variables qui sont des coordonnees de siconc (car elles sont aussi des coordonnees de variables de sortie)
    #
    coordinates = fh.variables['siconc'].getncattr('coordinates')
    coordlist = coordinates.split()
    for v_name, varin in fh.variables.items():
        for coordinate in coordlist:
            if (v_name == coordinate) and ( v_name != 'time'):
                outVar = f.createVariable(v_name, varin.datatype, varin.dimensions)
                # outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
                outVar[:] = varin[:]
    #
    sicbins = f.createVariable('siconcbin',np.float32,(binname,monname,dimlist[0],dimlist[1]))
    sicbins.missing_value = spval
    sicbins.units = units
    sicbins.standard_name = stdname
    sicbins.long_name = longname
    sicbins.coordinates = coordinates
    #
    swls = f.createVariable('Limit',np.float32,(levelname,monname,dimlist[0],dimlist[1]))
    swls.missing_value = spval
    swls.units = 'K'
    swls.standard_name = 'Limit'
    swls.long_name = 'Limit'
    swls.coordinates = coordinates
    #
    months[:] = month[:nmon]
    tbins[:] = Tbincen[:]
    levs[:] = levels[:]
    sicbins[:,:,:,:] = sicbin[:,:,:,:]
    swls[:,:,:,:] = swl[:,:,:,:]
    #
    f.close()
    #
    fh.close()
    fs.close()

['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CanESM5', 'FGOALS-f3-L', 'FIO-ESM-2-0', 'GFDL-CM4', 'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-MM']
Model : 0 BCC-CSM2-MR
  GSAT reference : 287.8904113769531
Model : 1 CAMS-CSM1-0
  GSAT reference : 287.0115966796875
Model : 2 CESM2


/tmp/ipykernel_96/3733691839.py:72: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  areacella = f.variables[champ][:,:]
/tmp/ipykernel_96/3733691839.py:106: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas = fh.variables[champ][it,:,:]
/tmp/ipykernel_96/3733691839.py:116: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  tas = fs.variables[champ][it,:,:]


  GSAT reference : 287.17376708984375


/tmp/ipykernel_96/3733691839.py:154: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  sicread = fh.variables["siconc"][it,:,:]
/tmp/ipykernel_96/3733691839.py:156: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  sicread = fs.variables["siconc"][it-nth,:,:]
/tmp/ipykernel_96/3733691839.py:236: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
  sicbins.missing_value = spval
/tmp/ipykernel_96/3733691839.py:243: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
  swls.missing_value = spval


Model : 3 CESM2-WACCM
  GSAT reference : 286.9439392089844
Model : 4 CNRM-CM6-1
  GSAT reference : 286.00653076171875
Model : 5 CNRM-CM6-1-HR
  GSAT reference : 285.5210266113281
Model : 6 CanESM5
  GSAT reference : 286.7170715332031
Model : 7 FGOALS-f3-L
  GSAT reference : 285.8725891113281
Model : 8 FIO-ESM-2-0
  GSAT reference : 286.4125671386719
Model : 9 GFDL-CM4
  GSAT reference : 285.9079284667969
Model : 10 GFDL-ESM4
  GSAT reference : 286.55810546875
Model : 11 INM-CM4-8
  GSAT reference : 286.4801940917969
Model : 12 INM-CM5-0
  GSAT reference : 286.3213806152344
Model : 13 IPSL-CM6A-LR
  GSAT reference : 285.9679260253906
Model : 14 MIROC-ES2L
  GSAT reference : 288.17913818359375
Model : 15 MIROC6
  GSAT reference : 288.4071044921875
Model : 16 MPI-ESM1-2-HR
  GSAT reference : 287.0749206542969
Model : 17 MRI-ESM2-0
  GSAT reference : 286.8822937011719
Model : 18 NESM3
  GSAT reference : 286.7835998535156
Model : 19 NorESM2-MM
  GSAT reference : 287.0330505371094


IndexError: index exceeds dimension bounds

## Conclusion

This notebook has produced new files representing sea ice as a function of global warming. We now have files for 19 different models, which means that 19 models will be used to study the evolution of sea ice.